In [ ]:
import string_similarity as fuzzy

# Demo of a custom text fuzzy-matching algorithm

## Setup and Preamble

In [ ]:
example_a = [
    "42 - 123 HelloWorld Ave, Waterloo ON, A2C 4E6", 
    "Unit 42 123 Hello World Avenue Waterloo ON A2C4E6"
]

Here is an illustrative example of the use case for this string-comparison algorithm, and some of the issues that should be addressed by the design:

In [ ]:
str.__eq__(example_a[0], example_a[1])

In just about any digital context, the two text strings from example_a are not equal. But, if asked whether or not those strings represent the same thing, or specifically in the case of the problem that motivated this algorithm 'do these two text representations refer to the same mailing address?' most human readers can say there is a high degree of confidence that they would in fact consider them as the same mailing address.

Algorithms such as taking the Jaccard Similarity (used for Power Query fuzzy-joins in Excel) offer a more granular approach to identifying similar text compared to the binary equal/not-equal operators without adding an excess of additional complexity due to being computationally simple and existing as an off-the-shelf solution. For those reasons it has been included as the reference for matching-performance.

My goal was to create a general-purpose tool that could accelerate an existing manual-review process for duplicate addresses between two systems where the digital representations were often not 'equal' in the trivial sense, but where there was a high incidence of duplicates that were just slightly off

In [ ]:
# naive equality, after stripping out spaces and standardizing letter-case
def heuristic_isequal(s1: str, s2: str): 
    return str.__eq__(
        s1.replace(" ", "").upper(), 
        s2.replace(" ", "").upper()
    )

# Jaccard Similarity = |intersection of characters| / |union of characters|
#  debatable whether the pre-processing of text to be standard improves the signal-to-noise, or hinders it for this algorithm
def jaccard_similarity(s1, s2):
    s1 = s1.replace(" ", "").upper()
    s2 = s2.replace(" ", "").upper()
    return round(len(set(s1).intersection(set(s2))) / len(set(s1).union(set(s2))), ndigits=3)

Example A: an example of several commonly-observed typos that prevent exact matching, yet a case where many humans share the intuition that they could very likely represent the same information (both being mailing addresses for the same physical residence)

In [ ]:
heuristic_isequal(example_a[0], example_a[1])

In [ ]:
jaccard_similarity(example_a[0], example_a[1])

In [ ]:
fuzzy.string_compare(example_a[0], example_a[1])

### How to read the output from string_compare: 

An output of 0 indicates absolutely no similarity, which is incredibly unlikely in practical cases.

A similarity score in the range (0.0, 0.6) indicates very low similarity, where manual reviewers would almost certainly conclude the text is not meaningfully the same.

Similarity equal to 1.0 means an exact match. Cases where this algorithm claims an exact match but a naive equality check does not is due to the basic pre-processing added to the function implementation for convenience of the intended application

Empirically, the boundary where the granularity of a 'clever' solution will separate from less sophisticated approaches is to reliably present ~0.90-0.99 when indicating "very similar, ususally with few minor typos", and <0.80 being the point where even if the input text has some indications of being the same, human reviewers would seriously question if it's a valid interpretation, relying heavily on other context to make a determination. Although arbitrary, it is worth noting that Excel's default fuzzy-match threshold is set to 0.80

**A word on pre-processing:** 
By default, string_compare strips out all spaces and ignores case/capitalization. 

For the purpose of characterising the algorithm performance, all further examples will not focus on pre-processing that could be uniformly applied to any method. 

(Note: choosing assumptions/business rules for preprocessing is an extremely important step, even before comparing or choosing algorithms)

Broadly speaking, the stated goal is that an inexact matching should implement the principle of "Pairs of text/strings that are more similar should score higher, and pairs that are less similar should score lower" such that the algorithm could sufficiently approximate an intuitive human judgement, allowing for increased automation of manual review processes.

The next section will be a series of examples consisting of three text strings, intended to check/demonstrate the characteristics of each inexact algorithm. 

The first two should be rated more similar to each other (positive match), but the first and third should be less similar (negative match), to demonstrate the level of granularity that algorithm is able to display.
Without imposing an arbitrary limit, the hope is to see "similar" pairings score >0.85 to be flagged for confirmation, and questionable 

In [ ]:
def run_test(triplet, fn=fuzzy.string_compare):
	# given a list of three strings, compare the first and second, then first and third using the provided function
	# standard/concise format for executing comparative examples
	print("similar pair: ", fn(triplet[0], triplet[1]), 
		" \nVS differing pair: ", fn(triplet[0], triplet[2]))

These are all convey roughly the same information to a natural-language reader, but the naive equality test fails to indicate that.

While it makes the point of how fragile exact matching is for comparing free-text entries, it does justify anything more sophisticated than some cleaning of the data according to business rules, and even the benchmark Jaccard method isn't necessary for this trivial example.

The remaining examples will be of an adversarial nature seeking (within the realm of plausible real-world data) to expose weaknesses in the algorithms, and determine the minimum level of complexity to be robust/reliable.


Example B

In [ ]:
example_b = [
    "Test String", 
    "TestString", 
    "test string"
]

In [ ]:
run_test(example_b, str.__eq__)

In [ ]:
run_test(example_b, heuristic_isequal)

In [ ]:
run_test(example_b, jaccard_similarity)

In [ ]:
run_test(example_b, fuzzy.string_compare)

## Demonstrate the algorithm characteristics

There were a series of guiding principals that motivated the design of a custom fuzzy-match algorithm. Specifically, there had to be characteristics that were desirable above and beyond the matching performance provided by any existing solution. Because otherwise, the practical solution would be to use an off-the-shelf tool that is more widely-known, and likely has implementations with better computational performance in the target environment. 

The original application was to be a VBA macro called in Excel workbooks, for use by an operational team of non-technical users on standard company-issued PCs. 

The original prototype was implemented in Julia for testing the compute-performance of the algorithm (due to the very low computational and dependency overhead of Julia compared to other high-level languages, the main cause of unexpectedly poor performance is poor algorithm design).

This Python implementation is a later translation for compatibility with potential new use cases.

**Design Considerations (DC)**

### DC I: 
Text (strings) that share most characters should be more similar, even if there is an ommission/addition between the pair

Example C

In [ ]:
example_c = [
    "Service Ontario, Waterloo Street S, Waterloo, ON", 
    "Srvc Ontario, Water Street W, Waterloo ON", 
    "TD Waterhouse Inc, Welland Crescent, Toronto, Ontario",
]

In [ ]:
run_test(example_c, fuzzy.string_compare)

In [ ]:
run_test(example_c, jaccard_similarity)

### DC II: 
Implicitly, text addresses of a similar length should end up more similar, even if there is substantial overlap in the set of characters. The set operations used for Jaccard similarity are largely blind to differences in length. Not only does string_compare account for the overlap in characters, but specifically the number of instances of matches.

Example D

In [ ]:
example_d = [
    "Quebec City JL International Airport, Québec City, Québec",
    "Quebec City Jean-Lesage Int'l Airport, Québec City, Québec",
    "Aéroport international Jacques Langlois de QQuuéebbeecc, Québec City Québec, Québec, Québec Canada"
]

In [ ]:
run_test(example_d, fuzzy.string_compare)

In [ ]:
run_test(example_d, jaccard_similarity)

### DC III: 
A 'bag-of-characters' approach like Jaccard without accounting for order is helpful in overlooking minor typos and ommissions (false-negative matches). Handling the strings as sets-of-characters is convenient, but that alone was not sufficiently sensitive for distinguishing actual differences (true-negative matches). 
To address this, beyond the number of matches found, the count of transpositions (matched characters that are out of order between the pair) is also factored into the final calculation of similarity
While the motivation included comparing addresses between systems that did not have the same delineation/standardization/formatting or content of fields, it was valid to assume the parts of the addresses between systems were going to be in the same/similar order within the strings supplied to the function (ie unit, street, city, postal code)

Example E

In [ ]:
example_e = [
    "Katherine", 
    "Kahterine", 
    "Kaetriannah"
]

In [ ]:
run_test(example_e, fuzzy.string_compare)

In [ ]:
run_test(example_e, jaccard_similarity)

### DC IV: 
Following the consideration of matching characters, length, and order, this motivated some concept of 'localized' inexact matching be implemented to better match the perspective of human readers of English. 

The final implementation included a method for acknowledging matching characters within similar/localized sections between strings, but also penalizing transpositions based on how many characters were out of order, and even disregarding matching characters if they were outside that localized section.

This concept of localized inexact matching turned out to be highly informative to the manual reviewers to contrast between minor typos and major differences, and justified the effort to implement the changes to their workflow. And because there was nothing else available to offer these attributes in the format required, that justified my efforts to implement it.

Example F

In [ ]:
example_f = [
    "the Sun life insurance company of Canada", 
    "Sun the life insurance company for Canada", 
    "Canada the Sun insurance company of life", 
]

In [ ]:
run_test(example_f, fuzzy.string_compare)

In [ ]:
run_test(example_f, jaccard_similarity)

### Utter Ridiculousness

Just how closely does this mimic the human intuition for what is the 'same' text, when the text has clearly been mangled just up to but not beyond the point of being unrecognizable? 

To be clear, string_compare is in no way an attempt to capture/compare semantic meaning, and does not see text with the same meaning as being highly similar after it has been run through a thesaurus. 

But if this algorithm can seem to cue off the same structural indicators of similarity as human readers in a case artificially-designed to play upon the selective auto-correct in the brains of fluent English-readers, I consider that a remarkably strong endorsement for this application. 

Example G

In [ ]:
example_g = [
    "According to research at Cambridge Univeristy, it doesn't matter in what order the letters in a word are, the only important thing is that the first and last letters be at the right places.",
    "Aoccdrnig to rseearch at Cmabrigde Uinervtisy, it deosn't mttaer in waht oredr the ltteers in a wrod are, the olny iprmoetnt tihng is taht the frist and lsat lteetrs be at the rghit pclaes.",
    "The rset can be a toatl mses and you can sitll raed it wouthit porbelm. Tihs is bcuseae the huamn mnid deos not raed ervey lteter by istlef, but the wrod as a wlohe. Wreid, rhgit?"
]

In [ ]:
run_test(example_g, fuzzy.string_compare)

In [ ]:
run_test(example_g, jaccard_similarity)

### Beyond Ridiculous

Especially as the length of the text goes up, and more of the alphabet and common punctuation marks appear in both strings, Jaccard similarity will tend to consider that all long strings are really quite similar. In fact, this is even accelerated by the pre-processing steps removing characters and case differences, because they reduce the total set of possible characters that could be differences. 

In all fairness, this is fully outside the reasonable or appropriate applications of the Jaccard method as a text-similarity measure, and entering into the realm of natural language processing where document-level analysis techniques make more practical sense. This is meant to stress-test the robustness of string_compare 

Extending that to its unnatural conclusion, even a relatively short string breaks the benchmark method, while string_compare is still able to make a clear distinction:

Example G

In [ ]:
example_h = [
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown foxes jumped over the lazy dog.",
    "And now for something completely different. Beelzebub proved to be quite the joker in this text circus."
]

In [ ]:
run_test(example_h, fuzzy.string_compare)

In [ ]:
run_test(example_h, jaccard_similarity)

## Try it!

The expectation is that a better-performing algorithm will rate more-similar pairs higher, and less-similar pairs lower. 
Further, a well-behaved similarity function will impose less of a penalty as 'less meaningful' changes are added between the compared texts, but changes that substantially change how the text address is interpreted should drastically lower the similarity score.

See if string_compare demonstrates this, based on texts that you consider to be more or less similar.

Is there an adversarial manipulation that can cause string_compare to behave in violation of human intuition about whether two strings are "similar" or not?

In [ ]:
text_1 = "Put your example text here"
text_2 = "Put our sample text there."
text_3 = "Enter something else here"

interactive_example = [text_1, text_2, text_3]

In [ ]:
run_test(interactive_example, fuzzy.string_compare)

In [ ]:
run_test(interactive_example, jaccard_similarity)

In [ ]:
run_test(interactive_example, heuristic_isequal)

Modify below code cell to customize string_compare parameters applied to the above inputs

In [ ]:
run_test(interactive_example,
	lambda t1, t2: fuzzy.string_compare(
		t1, t2, 
		strip=[" ", "-", ",", "."], 
		keep_case=True, 
		ignore_short=5 
	)
)

# The "So What?"

The above sections sought to establish the implementation of string_compare as substantively more refined than other available tools (while fitting within the same strict confines of the enterprise business environment). 

TLDR: There was an existing business process that was greatly accelerated by having such a solution available, the original sophisticated solution had become unavailable, and the most recent simple tool was proving insufficient. 

## Foundation

Delving into that application will yield the motivations of the current specification, but first it is worth highlighting the conceptual foundation of this project (Jaro-Winkler similarity https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance) which brought it to my attention from among other common string distance functions: https://en.wikipedia.org/wiki/String_metric 

Matthew A Jaro (1989) - "Advances in Record-Linkage Methodology as Applied to Matching the 1985 Census of Tampa, Florida"

To evaluate the coverage of the US Census against an independent and more rigourous survey of Tampa Florida, Jaro needed to more reliably identify links between the two sets of records, which were critically dependent on free text fields (ie names) as the main identifiers. The implemented solution calculated a distance considering matching instances of characters between strings, transposed matches, and limited the substring search range to half the length of the longer string.

William E Winkler (1990) - "The State of Record Linkage and Current Research Problems"

Winkler proposed an extension to Jaro's distance metric (as well as its application to record-matching) with an additional component to emphasize a common prefix between the texts (ie Katherine and Kathy have a common prefix of four characters, increasing the final similarity score)

## Application 

In the case of a jointly-owned account, both owners must periodically receive certain disclosures/updates/statements, which at the time were mailed out to a street address. Often, joint-owners live at the same mailing address and share finances, and for the purpose of cutting expenses (and providing less clutter in the Client experience) a single copy/envelope is sent for both addressees. However, it is also common for two joint owners not to share the same residence, either due to a separated couple, or accounts that might be jointly held for other reasons (ie an RESP started with contributions from a grand-parent jointly held with one or more parents of the beneficiary child). 

If each owner became a Client of Sun Life independently at different times, the mailing address associated with each individual may have been entered differently even if they were/are referring to the same address. Additionally, there is a need to check an individual's address between systems (ie Retail vs Group) for a routine reconciliation that ensures both systems are up to date/consistent across all experiences a Client has with Sun Life.

**Task:**

Between complete addresses (unit/apt/suite, PO Box, street number, street name, postal code, city, province) in different formats and potentially inconsistent free-text entry, determine if the residence/mailing address would be the same, or not

**Outcomes:** 

True-Positive - addresses are actually the same, condense documents to reduce costs and improve Client experience
True-Negative - two different addresses require multiple mailed copies to meet regulatory requirements for disclosure 
False-Negative - two addresses are actually the same, but are indicated to be different; two identical copies get mailed to the same residence, but all regulatory requirements are met 
False-Positive - two addresses which are actually different are flagged as being the same, and only one mailing address receives the necessary documents -> failure to meet regulatory mandate

**Existing Solution:** 

Concatenating all the applicable elements of the address into one string creates a high level of consistency between systems, and allows for a trivial equality comparison between addresses; every Positive indication was a True-Positive, meaning the risk of False-Positives was negated.

Further, all of the Negative-matches were flagged as Negatives, which is desirable. The issue is that there was a very high rate of False-Negatives, where a true match was not flagged.

To refine the classification, on a routine basis all the flagged non-matches were manually reviewed to see if there were any inexact matches that could be fixed and combined for mailing. A persistent pattern of very close matches emerged, of a sufficiently-high volume that combining them justified the person-hours for review/reconciliation (estimated 4-8 hrs, monthly), with the volume of Clients in the systems steadily increasing over time. Contributing to the tedium of this task was the fact that 99% of Negative pairs were 'not at all similar' or 'obviously the same' except for a few characters difference. 

The previous solution to automate this process was to submit each list of addresses via access to a Google Maps API to return the longitude/latitude co-ordinates of the full-text address, and determining address similarity based on the pair of numeric co-ordinates. When that option was no longer available, a new design was required.


# Algorithm Design In-Depth

There are theoretical limitations to the robustness of the logic, and perhaps also quirks of the current Python implementation, as well as a need for computational performance benchmarking before the point of considering further applications.

The range for local matches is determined as rounding down (square-root (length of the longer string) ) 
For two equal-length strings of length n: 
    worst-case of zero matching characters ~= n * sqrt(n) comparisons 
    best case of exactly matching strings = n comparisons


Explore or evaluate in more detail by enabling the verbose details

In [ ]:
text_1v = "Put your example text here"
text_2v = "Put our sample text there."
text_3v = "Enter something else here"

interactive_example_v = [text_1v, text_2v, text_3v]

In [ ]:
run_test(interactive_example_v,
	lambda t1, t2: fuzzy.string_compare(
		t1, t2, 
		#strip=[" "], 
		#keep_case=False, 
		#ignore_short=4,
        verbose=True 
	)
)

Alternative use of (1 - similarity) as a distance *metric*

Note formal axioms of a *metric*:  
- Dist between item a and itself = 0 (satisfied)  
- Symmetry of Dist(a, b) = Dist(b, a) (satisfied) 
- Triangle inequality Dist(a, b) + Dist(b, c) >= Dist(a, c) (satisfied)

So it counts as a metric, though as argued by Amos Tversky ("Features of Similarity", 1977) that is not terribly important for determining a useful quantification of similarity.

In [ ]:
def string_distance(s1, s2, 
		strip=[" "], 
        keep_case=False, 
        ignore_short=4, 
		verbose=False): 
    return 1 - fuzzy.string_compare(
		s1=s1, s2=s2, 
		strip=strip, 
		keep_case=keep_case, 
		ignore_short=ignore_short, 
        verbose=verbose 
	) 